In [1]:
from time import sleep
import requests 
from bs4 import BeautifulSoup
from selenium import webdriver

browser = webdriver.Firefox(executable_path=r'geckodriver-v0.26.0-win64\geckodriver.exe')
browser.get('https://web.whatsapp.com')

In [ ]:
url = "https://news.google.com/?hl=en-IN&gl=IN&ceid=IN:"
root = "https://news.google.com"
language_codes = {"english" : "en" , "hindi" : "hi"  , "marathi" :"mr" , "tamil":"ta" , "telugu":"te"}
understandable = [ ["show" , "news"] , ["send" , "news"] , ["show" ,"headlines"] , ["send" , "headlines"] , ["todays" , "news"] , ["today" , "news"] , ["today's","news"] ,["todays" , "headlines"] , ["today" , "headlines"] , ["today's","headlines"]]
def getNews(language) :
    text_box = browser.find_element_by_class_name("_3u328")
    response = "Let me fetch news:\n"
    text_box.send_keys(response)
    soup = BeautifulSoup( requests.get(url+language).content , "html5lib" )
    articles = soup.find_all('article' , class_ = "MQsxIb xTewfe R7GTQ keNKEd j7vNaf Cc0Z5d EjqUne")
    news = [ i.find_all('a' , class_="DY5T1d")[0].text for i in articles[:5]]
    links = [root+i.find('a')['href'][1:] for i in articles[:5]]
    links = [requests.get("http://thelink.la/api-shorten.php?url="+link).content.decode() for link in links]
    text_box.send_keys("*Headlines*\n")
    for i in range(5) :
        text_box.send_keys(news[i] + " ==> " + links[i] + "\n")
               
bot_users = {} #Active users 
while True : 
    unread = browser.find_elements_by_class_name("_0LqQ") #Greed dot 
    #unread = browser.find_elements_by_class_name("OUeyt") #Greed dot
    name , message =  '' ,'' 
    #print(unread[0] , " unread ")
    if len(unread) > 0 :
        ele = unread[-1] 
        action = webdriver.common.action_chains.ActionChains(browser)
        action.move_to_element_with_offset(ele , 0 , -40) # move a little left from green dot
        
    try :
        action.click()
        action.perform()
        action.click()
        action.perform()
    except Exception as e:
        pass
    
    
    try :
        #name = browser.find_element_by_class_name("_2zCDG").text  # name 
        name = browser.find_element_by_class_name("_3H4MS").text  # name 
        message = browser.find_elements_by_class_name("_F7Vk")[-1] #message
        print("name = " , name , "message = " , message.text)
        if 'activate' in message.text.lower() and 'bot' in message.text.lower() :
            if name not in bot_users : 
                bot_users[name] = True
                text_box = browser.find_element_by_class_name("_3u328")
                response = "Hi " + name + ". Abhishek's Bot here :). Now I am activated for you \n \n"
                text_box.send_keys(response)
        
                
        
        if name in bot_users :
            user_asking_for_news = False
            for cmd in understandable :
                if cmd[0] in message.text.lower() and cmd[1] in message.text.lower():
                    language = "en"
                    for lang in language_codes :
                        print(lang , "lang")
                        if lang in message.text.lower() :
                            language = language_codes[lang]
                            break
                    getNews(language)
                    print("daada" , cmd)
                    break
                    
        if 'deactivate' in message.text.lower() :
            if name in bot_users :
                text_box = browser.find_element_by_class_name("_3u328")
                response = "Bye "+name+".\n"
                text_box.send_keys(response)
                del bot_users[name]
    except Exception as e :
        print(e)
        pass 
    sleep(1)